<a href="https://colab.research.google.com/github/ahmedmabrouk24/Nueral-Network/blob/main/NN_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from zipfile import ZipFile

#with ZipFile('drive/MyDrive/NN Dataset.zip') as zipObj:

 # zipObj.extractall('drive/MyDrive/data')

In [2]:
pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ipyplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import cv2
import random
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import ipyplot

Instructions for updating:
non-resource variables are not supported in the long term



        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [6]:
def convert_all_image_to_jpg():
    for img in tqdm(os.listdir(TRAIN_DIR)):
        x = img.split(".")
        path = 'drive/MyDrive/data/Train/' + img
        name = 'drive/MyDrive/data/Train/' + x[0]
        im = Image.open(path)
        im.convert('RGB').save(name + ".jpg")
        if (x[1] != 'jpg'):
             os.remove(path) 



In [7]:
TRAIN_DIR = 'drive/MyDrive/data/Train'
TEST_DIR = 'drive/MyDrive/data/Test'
train_images=[]
test_images=[]
IMG_SIZE = 50
num_classes = 6
MODEL_NAME = 'sports_classification-cnn'

In [8]:
def augmentation(input_img):

    list_data = []

    #Horizontal Flip
    hflip= iaa.Fliplr(p=1.0)
    input_hf= hflip.augment_image(input_img) 
    list_data.append(input_hf)

    #Crop
    crop1 = iaa.Crop(percent=(0, 0.38)) 
    input_crop1 = crop1.augment_image(input_img)
    list_data.append(input_crop1)

    #shearing (random rotate)
    rot1 = iaa.Affine(rotate=(-70,20))
    input_rot1 = rot1.augment_image(input_img)
    list_data.append(input_rot1)

    rot2 = iaa.Affine(rotate=(70,20))
    input_rot2 = rot2.augment_image(input_img)
    list_data.append(input_rot2)

    #noise
    noise=iaa.AdditiveGaussianNoise(10,40)
    input_noise=noise.augment_image(input_img)
    list_data.append(input_noise)

    #elistic
    elastic = iaa.ElasticTransformation(alpha=2.0, sigma=0.5)
    input_elastic = elastic.augment_image(input_img)
    list_data.append(input_elastic)
  
    return list_data

In [9]:
    
def create_label(image_name):
    word_label = image_name[0:4]
    if word_label == 'Bask':
        return np.array([1,0,0,0,0,0])
    elif word_label == 'Foot':
        return np.array([0,1,0,0,0,0])
    elif word_label == 'Rowi':
        return np.array([0,0,1,0,0,0])
    elif word_label == 'Swim':
        return np.array([0,0,0,1,0,0])
    elif word_label == 'Tenn':
        return np.array([0,0,0,0,1,0])
    elif word_label == 'Yoga':
        return np.array([0,0,0,0,0,1])   

In [10]:
def create_train_data():

    training_data = []

    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,0)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        
        label_list = create_label(img)
        training_data.append([np.array(img_data), label_list])

        #append augmentation to train data
        aug_list = augmentation(img_data);
        for aug in aug_list:
            training_data.append([np.array(aug), label_list])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [11]:
if (os.path.exists('train_data.npy')):
    all_images = np.load('train_data.npy',allow_pickle=True)
else: 
    all_images = create_train_data()

for i in range(0 , 10000):
  train_images.append(all_images[i]);

for i in range(10000 , len(all_images)):
  test_images.append(all_images[i]);

X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = [i[1] for i in train_images]
X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = [i[1] for i in test_images]

100%|██████████| 1681/1681 [00:29<00:00, 56.45it/s]
/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [12]:
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
conv1 = conv_2d(conv_input, 16, 5, activation='relu')
pool1 = max_pool_2d(conv1, 5)

conv2 = conv_2d(pool1, 32, 5, activation='relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool2, 32, 5, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 32, 5, activation='relu')
pool4 = max_pool_2d(conv4, 5)

conv5 = conv_2d(pool4, 16, 5, activation='relu')
pool5 = max_pool_2d(conv5, 5)

fully_layer = fully_connected(pool5, 1024, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)
print (X_train.shape)

if (os.path.exists('model.tfl.meta')):
    model.load('./model.tfl')
else:
    model.fit({'input': X_train}, {'targets': y_train}, n_epoch=10,
          validation_set=({'input': X_test}, {'targets': y_test}),
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    model.save('model.tfl')

Training Step: 1569  | total loss: 1.28032 | time: 39.613s
| Adam | epoch: 010 | loss: 1.28032 - acc: 0.4802 -- iter: 09984/10000
Training Step: 1570  | total loss: 1.28342 | time: 40.875s
| Adam | epoch: 010 | loss: 1.28342 - acc: 0.4743 | val_loss: 1.27551 - val_acc: 0.4992 -- iter: 10000/10000
--
